¿Cuál modelo es mejor?




<img src="three_models.png" alt="Drawing" style="width:60%;"/>

¿Qué comportamiento tienen un modelo de alto bias vs un modelo de alta varianza en train? ¿Y en test?

Entre un pantalón que me queda chico y uno que me queda grande, ¿cuál elijo? 

Elegimos el que queda grande y buscamos soluciones, como un cinturón.

Vamos a buscar sobreajustar el modelo para luego usar distintas técnicas que nos permiten controlar el overfitting.

Intuición: ¿Cuándo hay más overfitting?

Pocas observaciones - Muchas observaciones

Pocos parámetros - Muchos parámetros 

Pocas epochs - Muchas epochs


<img src="masteryoda.jpeg" alt="Drawing" style="width:70%;"/>

## Cantidad de parámetros

In [ ]:
import numpy as np
from matplotlib import pyplot as plt 
%matplotlib inline
from IPython.display import HTML
import reg_helper as RHelper
import draw_nn


In [ ]:

X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy') 
y_test = np.load('y_test.npy')
print('Cantidad de observaciones:')
print('Train',X_train.shape[0])
print('Test',X_test.shape[0])
print('Dos variables de entrada. Dos clases de salida (binario)')
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,6))
RHelper.plot_boundaries(X_train, y_train, ax=ax1)
RHelper.plot_boundaries(X_test, y_test, ax=ax2)
plt.show()

**Regresión logística polinomial**

El logit es una ecuación polinómica de las dos entradas. La cantidad de parámetros a ajustar depende del grado del polinomio.

\begin{equation}
\large
a = w_0 + x_1w_1 + x_2w_2 + x_1x_2w_3 + w_4x_1^2 + w_5x_2^2 + ... + w_Nx_1^K
\end{equation}
\begin{equation}
\large
y = \sigma(a)
\end{equation}


Ajustemos una recta (polinomio de grado 1)

In [ ]:
tr_acc, ts_acc, coefs = RHelper.fit_and_get_regions(X_train, y_train, X_test, y_test, degree=1)
print('Acurracy en train', tr_acc)
print('Accuracy en test', ts_acc)
print('Cantidad de parámetros', coefs.shape[1])


Ajustamos una cuadrática...

In [ ]:
tr_acc, ts_acc, coefs = RHelper.fit_and_get_regions(X_train, y_train, X_test, y_test, degree=2)
print('Acurracy en train', tr_acc)
print('Accuracy en test', ts_acc)
print('Cantidad de parámetros', coefs.shape[1])


In [ ]:
tr_acc, ts_acc, coefs = RHelper.fit_and_get_regions(X_train, y_train, X_test, y_test, degree=6)
print('Acurracy en train', tr_acc)
print('Accuracy en test', ts_acc)
print('Cantidad de parámetros', coefs.shape[1])


In [ ]:
tr_acc, ts_acc, coefs = RHelper.fit_and_get_regions(X_train, y_train, X_test, y_test, degree=18)
print('Acurracy en train', tr_acc)
print('Accuracy en test', ts_acc)
print('Cantidad de parámetros', coefs.shape[1])


¿En qué modelo empezamos a sobreajustar? ¿Qué harían para elegir el mejor modelo?

In [ ]:
options = [{'degree': 1, 'lambd': 0}, 
           {'degree': 2, 'lambd': 0}, 
           {'degree': 3, 'lambd': 0}, 
           {'degree': 4, 'lambd': 0}, 
           {'degree': 5, 'lambd': 0}, 
           {'degree': 6, 'lambd': 0}, 
           {'degree': 7, 'lambd': 0}, 
           {'degree': 8, 'lambd': 0}, 
           {'degree': 9, 'lambd': 0}, 
           {'degree': 10, 'lambd': 0},
           {'degree': 11, 'lambd': 0},
           {'degree': 12, 'lambd': 0},
           {'degree': 13, 'lambd': 0},
           {'degree': 14, 'lambd': 0}, 
           {'degree': 15, 'lambd': 0}, 
           {'degree': 16, 'lambd': 0}, 
           {'degree': 17, 'lambd': 0}, 
           {'degree': 18, 'lambd': 0}
           ]
degrees, lambdas, train_acc_array, test_acc_array, coefs_array_mean, coefs_array_std, coefs_abs_max, coefs_norm, coefs_num = RHelper.test_options(X_train, y_train, X_test, y_test, options, plot_it=False)

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(degrees, train_acc_array, label="Train")
ax.plot(degrees, test_acc_array, label="Test")
plt.title("Accuracies")

plt.xlabel('Orden del polinomio')
plt.ylabel('Accuracy')
ax.set_xticks(degrees)
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.plot(degrees, coefs_num)
ax.legend()
plt.xlabel('Orden del polinomio')
plt.ylabel('Cantidad de parámetros')
ax.set_xticks(degrees)
plt.show()

In [ ]:
plt.figure(figsize=(20,5))
plt.ylabel("Norma de vector de pesos")
plt.plot(degrees, coefs_norm)
plt.xlabel('Orden del polinomio')
plt.legend()
plt.show()

## Regularización

Recordemos la función de costo de entropía cruzada para una observación:

$Loss = - \sum_i [p_i  \log(\hat{p}_i) + (1-p_i) \log(1-\hat{p}_i)]$  

donde: 
- $p_i$ solo puede valer 1 o 0. Vale 1 si pertenece a la clase $i$'esima, y 0 si no pertenece
- $\hat{p}_i$ es la estimación de la probabilidad de que $X_i$ pertenezca a la clase. Por ejemplo, la sigmoidea de la ecuación polinómica de pesos

La regularización busca conseguir modelos más generalizables, es decir, con menor sobreajuste. Esto se logra mediante modelos con pesos bajos, que se obtienen penalizando los pesos altos. Para ello, se agrega un término a la función de costo que es función del módulo de los pesos.

$Loss = - \sum_i [p_i  \log(\hat{p}_i) + (1-p_i) \log(1-\hat{p}_i)] + f(\mathbf {w})$  

------------------------------------------------------------------------------------------------------------

**LASSO**: normalización de norma L1

$Loss = - \sum_i [p_i  \log(\hat{p}_i) + (1-p_i) \log(1-\hat{p}_i)] + \lambda  ||\mathbf {w}||_1$ 

$||\mathbf {w}||_1 = |w_1|+|w_2|+...+|w_n|$

<img src="lasso_penalty.png" alt="Drawing" style="width:30%;"/>

La disminución en el valor de loss depende de cuánto cambie el valor de un peso, pero no depende del valor del peso. 

------------------------------------------------------------------------------------------------------

**RIDGE**: normalización de norma L2

$Loss = - \sum_i [p_i  \log(\hat{p}_i) + (1-p_i) \log(1-\hat{p}_i)] + \lambda   ||\mathbf {w}||_2$ 

$||\mathbf {w}||_2 = \sqrt{w_1^2+w_2^2+...+w_n^2}$

<img src="ridge_penalty.png" alt="Drawing" style="width:30%;"/>


La disminución en el valor de loss será distinta según el valor del peso 

## Práctica con Pesos y Alturas

In [ ]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/rn-2019-itba/Clase-4---LDA---QDA---RL---DT---RF/master/data/alturas-pesos-mils-train.csv')
data = df[['Altura', 'Peso']].values
y=np.array([0 if x=='Hombre' else 1 for x in df['Genero']])

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data,y, test_size=0.4, shuffle=True, random_state=500)


In [ ]:
clf_sin_regu = LogisticRegression()
clf_sin_regu.fit(X_train,y_train)

In [ ]:
X_train.shape

In [ ]:
fig = plt.figure(figsize=(15,15))

plt.scatter(data[y==0,0],data[y==0,1], marker='+')
plt.scatter(data[y==1,0],data[y==1,1], c= 'green', marker='o')
w = clf_sin_regu.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(min(data[:,0]),max(data[:,0]), 100)
yy = a * xx - (clf_sin_regu.intercept_[0]) / w[1]

plt.plot(xx, yy, 'k-')

In [ ]:
def plotLinearBoundary(X,y,clf):
    fig = plt.figure(figsize=(15,15))
    plt.scatter(X[y==0,0],X[y==0,1], marker='+')
    plt.scatter(X[y==1,0],X[y==1,1], c= 'green', marker='o')
    w = np.squeeze(clf.coef_)
    a = -w[0] / w[1]
    x0 = np.linspace(min(X[:,0]),max(X[:,0]), 100)
    x1 = a * x0 - (np.squeeze(clf.intercept_)) / w[1]

    plt.plot(x0, x1, 'k-')

In [ ]:
clf_lasso = Lasso(alpha=0.1)
clf_lasso.fit(X_train,y_train)
plotLinearBoundary(data,y,clf_lasso)


In [ ]:
clf_ridge = Ridge(alpha=0.1)
clf_ridge.fit(X_train,y_train)

plotLinearBoundary(data, y, clf_ridge)


In [ ]:
def plotBoundary(data, labels, clf_1, N=300,degree=False,include_bias=True):
    class_1 = data[labels == 1]
    class_0 = data[labels == 0]
    mins = data[:,:2].min(axis=0)
    maxs = data[:,:2].max(axis=0)
    x1 = np.linspace(mins[0], maxs[0], N)
    x2 = np.linspace(mins[1], maxs[1], N)
    x1, x2 = np.meshgrid(x1, x2)
    X=np.c_[x1.flatten(), x2.flatten()]
    if degree is not None:
        poly=PolynomialFeatures(degree,include_bias=include_bias)
        X=poly.fit_transform(X)
    Z_nn = clf_1.predict(X)#[:, 0]

    # Put the result into a color plot
    Z_nn = Z_nn.reshape(x1.shape)
    
    fig = plt.figure(5,figsize=(15,15))
    ax = fig.gca()
    cm = plt.cm.RdBu
        
    ax.contour(x1, x2, Z_nn, (0.5,), colors='black', linewidths=1)
    ax.scatter(class_1[:,0], class_1[:,1], c= 'green', marker='o', s=20, alpha=0.5)
    ax.scatter(class_0[:,0], class_0[:,1], c= 'blue', marker='+', s=20, alpha=0.5)
    plt.show()

In [ ]:

from sklearn.preprocessing import PolynomialFeatures
order = 4
poly = PolynomialFeatures(order,include_bias=True)
X_poly_train =poly.fit_transform(X_train)
lasso_poly = Lasso(alpha=0.8)
lasso_poly.fit(X_poly_train,y_train)
print("Cantidad de features seleccionadas: ", np.sum(lasso_poly.coef_!=0), " de ", len(lasso_poly.coef_))
plotBoundary(data, y, lasso_poly, 1000,degree=order)

In [ ]:
ridge_poly = Ridge(alpha=0.8)
ridge_poly.fit(X_poly_train,y_train)
print("Cantidad de features seleccionadas: ", np.sum(ridge_poly.coef_!=0), " de ", len(ridge_poly.coef_))
plotBoundary(data, y, ridge_poly, 1000,degree=order)

**Red neuronal multicapa**

In [ ]:
input_shape = 2
hidden_units = 20 
output_size = 1
network = draw_nn.DrawNN( [input_shape, hidden_units, output_size] )
network.draw()

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
def get_two_layer_model_compiled(input_shape, output_size, hidden_units, lr=2,decay=0.0):

    model = Sequential()
    sgd = optimizers.SGD(lr=lr, decay=decay)
    model.add(Dense(hidden_units,input_dim=input_shape,  activation='sigmoid', ))
    model.add(Dense(output_size, 
                    activation='sigmoid', 
                    kernel_initializer='zeros', 
                    name='Salida'
                   ))
    model.compile(loss = 'binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [ ]:
from fnn_helper import PlotLosses
from keras import optimizers

lr=1
decay=0

plot_losses = PlotLosses(plot_interval=200, evaluate_interval=None, x_val=X_test, y_val_categorical=y_test)
two_layer_model = get_two_layer_model_compiled(input_shape, 
                                                 output_size, 
                                                 hidden_units, lr, decay 
                                                )
two_layer_model.summary()

In [ ]:

epochs = 10

hist = two_layer_model.fit(X_train, 
          y_train,
          batch_size=5,
          epochs=epochs, 
          verbose=1, 
          validation_data=(X_test, y_test), 
          #callbacks=[plot_losses],
         )

print('Cantidad de parámetros',two_layer_model.count_params())
print('Mejor acc en train', max(hist.history['acc']))
print('Mejor acc en test', max(hist.history['val_acc']))

In [ ]:
hidden_units = 2 
two_layer_model = get_two_layer_model_compiled(input_shape, 
                                                 output_size, 
                                                 hidden_units=hidden_units, 
                                                 lr=lr, 
                                                 decay=decay,
                                                )
two_layer_model.summary()

In [ ]:
hist = two_layer_model.fit(X_train, 
          y_train, batch_size = batch_size,
          epochs=epochs, 
          verbose=0, 
          validation_data=(X_test, y_test), 
          #callbacks=[plot_losses],
         )
print('Cantidad de parámetros',two_layer_model.count_params())
print('Mejor acc en train', max(hist.history['acc']))
print('Mejor acc en test', max(hist.history['val_acc']))

In [ ]:
hidden_units = 200 
two_layer_model = get_two_layer_model_compiled(input_shape, 
                                                 output_size, 
                                                 hidden_units=hidden_units, 
                                                 lr=lr, 
                                                 decay=decay,
                                                )

In [ ]:
hist = two_layer_model.fit(X_train, 
          y_train, batch_size = batch_size,
          epochs=epochs, 
          verbose=0, 
          validation_data=(X_test, y_test), 
          callbacks=[plot_losses],
         )
print('Cantidad de parámetros',two_layer_model.count_params())
print('Mejor acc en train', max(hist.history['acc']))
print('Mejor acc en test', max(hist.history['val_acc']))

## Tamaño del dataset

**Cross-Validation**

In [ ]:
#dataset_200 = np.load('200_samples_OK.npy')
dataset_200 = np.load('1000_samples.npy')

X = dataset_200[:,:2]
y = dataset_200[:, 2]
f = plt.figure(figsize=(20,6))
RHelper.plot_boundaries(X, y)
plt.show()

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(10)
splited_indexs = skf.split(X, y)
print(type(splited_indexs))
print(next(splited_indexs))


In [ ]:
i=0
training_sets = []
for train_index, test_index in splited_indexs:
    i=i+1
    print("CV dataset:", i)
    print(train_index.shape, test_index.shape)
    dictionary = {'X_train':X[train_index], 'y_train':y[train_index], 'X_test':X[test_index],'y_test':y[test_index]}
    training_sets.append(dictionary)

In [ ]:
idx = 3
RHelper.plot_boundaries(training_sets[idx]['X_train'],training_sets[idx]['y_train'])
plt.title('Train subset in fold ' + str(idx))
plt.show()
plt.title('Test subset in fold ' + str(idx))
RHelper.plot_boundaries(training_sets[idx]['X_test'],training_sets[idx]['y_test'])
plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score, train_test_split
from keras.wrappers.scikit_learn import KerasClassifier
classifier = KerasClassifier(build_fn=get_two_layer_model_compiled(input_shape, 
                                      output_size, 
                                      hidden_units), batch_size=25,  
                             epochs=10, verbose=0)
cross_val_scores = cross_val_score(estimator=classifier, X=X, y=y, cv=skf, 
                                   scoring='accuracy')

## Regularización en redes neuronales
**Regularizers de Keras**

En una red neuronal, la regularización se define en cada capa. Puedo definir los pesos de cuáles capas se sumarán en la función de costo. Usando Keras, puedo definir el 'regularizer' de cada capa. 
Puedo asignar un regularizer como 'kernel_regularizer', 'bias_regularizer', o 'activity_regularizer'. El que hemos explicado hasta ahora, es decir, penalizando los pesos altos, es el kernel_regularizer. El bias regularizer penaliza un alto bias, y el activity_regularizer penaliza un total alto de la ecuacion $wx+b$.

In [ ]:
from keras import regularizers
#Los que ya están implementados en Keras son: 

regularizers.l1(0.)
regularizers.l2(0.)
regularizers.l1_l2(l1=0.01, l2=0.01)


In [ ]:
model = Sequential()
model.add(Dense(64, input_dim=64,
                kernel_regularizer=regularizers.l2(0.01))

**Dropout**

La técnica de dropout consiste en ignorar ciertas unidades de la red durante el entrenamiento. 

<img src="dropout.png" alt="Drawing" style="width:60%;"/>

Esto se consigue asignando un peso nulo a la conexión entre esta unidad y la siguiente capa (ignorarla en el *foward propagation*) y un peso nulo a su conexiones con las unidades de la capa anterior (ignorarla en el *backpropagation*). La decisión de si una unidad es ignorada o no en cada iteración del entrenamiento es aleatoria: se define como una probabilidad $p$. Este es el único hiperparámetro del dropout. Cada nodo individual tiene una probabilidad $p$ de ser conservado (es decir, $1-p$ de ser ignorado).

Durante la etapa de testeo, se usan todos los nodos pero se reducen sus pesos en un factor $p$.

El dropout en Keras se agrega como una capa de tipo Dropout, que se aplica a las conexiones entre la capa inmediatamente anterior e inmediatamente posterior. 

In [ ]:
from keras.layers import Dropout
model = Sequential()
model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(30, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
# Compile model
sgd = optimizers.SGD(lr=0.1)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()